# Arbol de decision
En este notebook vamos a entrenar y explorar un arbol de decision.

In [29]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing, tree
from ipywidgets import Button, IntSlider, interactive

from preprocessing import cargarDatasets
from preprocessing import prepararSetDeEntrenamiento
from preprocessing import prepararSetDeValidacion
from preprocessing import ingenieriaDeFeauturesArboles1

from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.model_selection import train_test_split,GridSearchCV

## Carga y preprocesamiento de los datos
Cargamos los datasets y los preparamos para usar.

In [30]:
train_df,final_df = cargarDatasets()
train_df = prepararSetDeEntrenamiento(train_df)
final_df = prepararSetDeValidacion(final_df)

In [31]:
X,y,df,y_encoder = ingenieriaDeFeauturesArboles1(train_df)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=117, test_size=0.1, stratify=y)

In [32]:
X_train.head()

,ganancia_perdida_declarada_bolsa_argentina,categoria_de_trabajo_empleadao_estatal,categoria_de_trabajo_empleado_municipal,categoria_de_trabajo_empleado_provincial,categoria_de_trabajo_monotibutista,categoria_de_trabajo_relacion_de_dependencia,categoria_de_trabajo_responsable_inscripto,categoria_de_trabajo_sin_trabajo,categoria_de_trabajo_trabajo_voluntariado,estado_marital_matrimonio_civil,...,trabajo_otros,trabajo_profesional_especializado,trabajo_reparador,trabajo_sector_primario,trabajo_seguridad,trabajo_servicio_domestico,trabajo_soporte_tecnico,trabajo_transporte,trabajo_ventas,educacion_alcanzada_encoded
29580,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,5
7359,0,0,1,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,8
30739,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,7
23107,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,0,8
14357,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,7


* ingenieria de feautures (2 formas)
* dividimos cada set en 2.
* entrenamos (2 arboles)
* metricas

fit: ajusta el modelo al set de entrenamiento
predict: que agarre un ejemplo y haga una prediccion (predice la clase a la que pertenece)


hiperparametros: quiero encontrar el mejor modelo para este dataset. Son parametros que el modelo no aprende, hay que decirselos.
-> random state: quiero que si un valor es random, que sea reproducible. Que el modelo vuelva a dar lo mismo cada vez que lo corro (que sea determinista).
->split: cuantos feautures miro para ir dividiendo.

Como el modelo es simple, le pasamos features que sabemos que van a andar bien.

In [33]:
params1 = {
    'max_depth': np.arange(1, 10),
    'min_samples_leaf': np.arange(10, 30),
    "criterion": ["gini", "entropy"],
}

arbol_1 = tree.DecisionTreeClassifier()
gscv1 = GridSearchCV(
    arbol_1, params1, scoring='accuracy', n_jobs=-1, cv=5, return_train_score=True
).fit(X_train, y_train)

arbol_1.fit(X_train, y_train)
y_pred1 = arbol_1.predict(X_test)
accuracy_score(y_test, y_pred1) 

0.8492477740251766

In [34]:
print(f"Best score: {gscv1.best_score_}")
print(f"Best params {gscv1.best_params_}")

Best score: 0.85486630256226
Best params {'criterion': 'entropy', 'max_depth': 9, 'min_samples_leaf': 19}


# plot del arbol 1

In [41]:
from sklearn.tree import export_graphviz
from pydotplus import graph_from_dot_data

feature_names = list(X_train.columns.values)

feature_names
#dot_data = export_graphviz(arbol_1, feature_names=feature_names)

#graph = graph_from_dot_data(dot_data)
#graph.write_png('tree.png')

['ganancia_perdida_declarada_bolsa_argentina',
 'categoria_de_trabajo_empleadao_estatal',
 'categoria_de_trabajo_empleado_municipal',
 'categoria_de_trabajo_empleado_provincial',
 'categoria_de_trabajo_monotibutista',
 'categoria_de_trabajo_relacion_de_dependencia',
 'categoria_de_trabajo_responsable_inscripto',
 'categoria_de_trabajo_sin_trabajo',
 'categoria_de_trabajo_trabajo_voluntariado',
 'estado_marital_matrimonio_civil',
 'estado_marital_matrimonio_militar',
 'estado_marital_pareja_no_presente',
 'estado_marital_separado',
 'estado_marital_sin_matrimonio',
 'estado_marital_viudo_a',
 'genero_mujer',
 'rol_familiar_registrado_casado',
 'rol_familiar_registrado_con_hijos',
 'rol_familiar_registrado_otro',
 'rol_familiar_registrado_sin_familia',
 'rol_familiar_registrado_soltero_a',
 'trabajo_directivo_gerente',
 'trabajo_ejercito',
 'trabajo_entretenimiento',
 'trabajo_inspector',
 'trabajo_limpiador',
 'trabajo_otros',
 'trabajo_profesional_especializado',
 'trabajo_reparador',


## Metricas
Las metricas a utilizar son:
* AUC-ROC  
* Matriz de confusión  
* Accuracy  
* Precisión  
* Recall  

Observamos las metricas obtenidas de las predicciones con el set de prueba.

### Curva AUC ROC
Primero calculamos el score obtenido con la curva de AUC ROC.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, plot_roc_curve, roc_auc_score

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred1)
display(fpr)
display(tpr)
display(thresholds)

In [ ]:
plot_roc_curve(arbol_1,X_test,y_test)
plt.show()

Podemos ver que la curva de auc roc nos indica que es mas probable que predigamos la clase de manera correcta.

### Matriz de confusion

In [ ]:
from sklearn.metrics import plot_confusion_matrix

fig, ax = plt.subplots(figsize=(15, 7))
plt.grid(False)
plot_confusion_matrix(
    arbol_1, X_test, y_test, cmap=plt.cm.Blues, display_labels=['1', '0'], ax=ax
)
plt.show()
#plot_confusion_matrix(y_test, arbol_1.predict_proba(X_test)[:,1]) HAY Q USAR PREDICT PROBA?

Podemos ver (por ahora, tal vez hay q cambiar algo) que estamos prediciendo de manera correcta a 2281 + 489 casos, y estamos prediciendo mal a 296 + 192.

obtenemos 192 falsos negativos 
295 falsos positivos.

### Accuracy
esta mas arriba, despues pasarlo aca y agregar conclusion

### Precision

In [ ]:
precision_score(y_test, y_pred1)

### Recall

In [ ]:
recall_score(y_test, y_pred1)

## Prediccion sobre el data set de hold out

In [ ]:
#no anda porque hay que convertir los datos pero planteo la idea
ho_prediction = arbol_1.predict(final_df)
final_df['tiene_alto_valor_adquisitivo'] = ho_prediction

In [ ]:
Dudas
otras formas de preprocesamiento?
estaria bien sacar algunas de las columnas de trabajo o categoria? el arbol es muy grande para graficar.